In [ ]:
!pip install gurobipy

     |████████████████████████████████| 11.5 MB 4.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp


from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error as mse


In [ ]:
df=pd.read_csv('training_data.csv')


In [ ]:
def generate_sigma_obj(df):
  x=df.drop(['y'],axis=1).values
  y=df['y'].values

  predictors=x.shape[1]

  x=np.concatenate([x,np.ones(shape=(x.shape[0],1))],axis=1)
  
  sigma=np.zeros(shape=(2*predictors+1,2*predictors+1))
  sigma[:predictors+1,:predictors+1]=x.T@x


  obj=np.zeros(
      shape=(2*predictors+1)
  )
  obj[:predictors+1]=-2*(y.T@x)

  return (sigma,obj)


In [ ]:
def prepare_constraint_matrix(K=10,M=20):
  global predictors

  A=np.zeros((2*predictors+1,
              2*predictors+1))
  b=np.zeros((2*predictors+1))

  sense=np.array(['']*(2*predictors+1))

  rw=0
  for i in range(predictors):
    A[rw,[i,i+predictors+1]]=[1,-M]#upper bound for bj <M*zj
    sense[rw]='<'

    A[rw+predictors,[i,i+predictors+1]]=[1,M]#lower bound bj >M*zj
    sense[rw+predictors]='>'
    rw+=1

  A[-1,predictors+1:]=[1]*predictors
  b[-1]=K
  sense[-1]='<'

  return (A,b,sense)

In [ ]:
def formulate_bounds_vtypes(M):
  global predictors
  ub=[M]*predictors +[M*1000] +[1]*predictors
  lb=[-M]*predictors +[-M*1000] +[0]*predictors

  vtype=['C']*(predictors+1) +['B']*(predictors)

  return (ub,lb,vtype)

In [ ]:
def optimize_loss(sigma,obj,A,b,sense,vtype,ub,lb):

  biMod = gp.Model()
  biMod_x = biMod.addMVar(len(obj),vtype=vtype,ub=ub,lb=lb) # vtype can be: 'C' or 'I' or 'B'
  biMod_con = biMod.addMConstrs(A, biMod_x, sense, b)
  biMod.setMObjective(sigma,obj,0,sense=gp.GRB.MINIMIZE)

  biMod.Params.OutputFlag = 0 
  biMod.optimize()

  return biMod

In [ ]:
def compute_loss(biMod,df_test,return_weight=False):
  global predictors
  x_test=df_test.drop(['y'],axis=1).values
  y_test=df_test['y'].values

  x_test=np.concatenate([x_test,np.ones(shape=(x_test.shape[0],1))],axis=1)

  y_pred=x_test[:,:predictors+1]@biMod.x[:predictors+1]
  err=mse(y_test,y_pred)

  if return_weight:
    return (err,biMod.x[:predictors+1])
  return err





In [ ]:
big_m=20
number_predictors_allowed=10
predictors=df.shape[1]-1


def perform_10fold_cv(number_predictors_allowed,big_m=20):
  k_fold=KFold(n_splits=10,shuffle=True)


  lt_mse=[]
  for train_idx,test_idx in k_fold.split(df):
    df_train=df.loc[train_idx]
    df_test=df.loc[test_idx]

    sigma,obj=generate_sigma_obj(df_train)
    A,b,sense=prepare_constraint_matrix(K=number_predictors_allowed,M=big_m)
    ub,lb,vtype=formulate_bounds_vtypes(M=big_m)
    biMod=optimize_loss(sigma,obj,A,b,sense,vtype,ub,lb)
    err=compute_loss(biMod,df_test,return_weight=False)

    lt_mse.append(err)
  return lt_mse




In [ ]:
dict_k={}

for k in np.arange(5,55,5):
  print(f'processing for {k} predictors','\n\n')

  temp_lt=perform_10fold_cv(number_predictors_allowed=k)
  dict_k[str(k)]=temp_lt

processing for 5 predictors 




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Mod

processing for 10 predictors 




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Mod

processing for 15 predictors 




...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Mod

processing for 45 predictors 




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Mod

processing for 50 predictors 




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Deprecated, use Model.addMConstr() instead


In [ ]:
pd.DataFrame(dict_k).mean(axis=0)#.to_csv('err.csv')

5     3.859177
10    2.872555
15    3.070032
20    3.284382
25    3.277699
30    3.428896
35    3.221228
40    3.196468
45    3.507588
50    3.289136
dtype: float64

# Constraints

In [ ]:
x=df.drop(['y'],axis=1).values
y=df['y'].values#.reshape(-1,1)

x=np.concatenate([x,np.ones(shape=(x.shape[0],1))],axis=1)

#We have an alzebraic formulation for all the predictors and intercept, we need to incorporate z based pivot variables in our loss function
#This needs to be done beacus eof the design of scipy optimization instance that requires variable to be of fixed length in our constraint and loss function

expanded_loss_term1=np.zeros(shape=(2*predictors+1,2*predictors+1))
expanded_loss_term1[:predictors+1,:predictors+1]=x.T@x


expanded_loss_term2=np.zeros(
    shape=(2*predictors+1)
)
expanded_loss_term2[:predictors+1]=-2*(y.T@x)

#Thus we have formulated two terms for our loss function

sigma=expanded_loss_term1
obj=expanded_loss_term2



In [ ]:
#Finding appropriate M by computing regression coefficients of our dataset

lt_coefs=[]

lr=LinearRegression()
lr.fit(x,y)
lt_coefs.extend(lr.coef_)

for i in range(x.shape[1]):
  lr=LinearRegression()
  lr.fit(x[:,i].reshape(-1,1),y)
  lt_coefs.extend(lr.coef_)


print(np.max(np.absolute(np.array(lt_coefs))))
# Our starting M can be--> 100 >>2.8*10
M=np.round(np.max(np.absolute(np.array(lt_coefs)))*10,-2)
M=20

bounds=[(0,1.1) for _ in range(51)]+[(-M,M) for _ in range(predictors)]


NameError: ignored

In [ ]:
#Big M constraints for every predictor
#2* preditors number of constraints
#Orientation of the decision variables - preditors + intercept + gates(decision variables)

#1 constraint for z number of variables

K=25

A=np.zeros((2*predictors+1,
            2*predictors+1))
b=np.zeros((2*predictors+1))

sense=np.array(['']*(2*predictors+1))

rw=0
for i in range(predictors):
  A[rw,[i,i+predictors+1]]=[1,-M]#upper bound for bj <M*zj
  sense[rw]='<'

  A[rw+predictors,[i,i+predictors+1]]=[1,M]#lower bound bj >M*zj
  sense[rw+predictors]='>'

  rw+=1

A[-1,predictors+1:]=[1]*50
b[-1]=K
sense[-1]='<'

ub=[M]*50 +[M*1000] +[1]*50
lb=[-M]*50 +[-M*1000] +[0]*50

vtype=['C']*(predictors+1) +['B']*50

NameError: ignored

In [ ]:
vtype=['C']*51 +['B']*50
# obj

In [ ]:
biMod = gp.Model()
biMod_x = biMod.addMVar(len(obj),vtype=vtype,ub=ub,lb=lb) # vtype can be: 'C' or 'I' or 'B'
# biMod_con = biMod.addMConstrs(A, biMod_x, sense, b)
# biMod.setMObjective(sigma,obj,0,sense=gp.GRB.MINIMIZE)

# biMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
# biMod.optimize()

AssertionError: ignored

In [ ]:
(x[:,:predictors+1]@biMod.x[:predictors+1]).shape

(250,)

In [ ]:
from sklearn.model_selection import train_test_split

df_train,df_test=train_test_split(df,train_size=0.8)

predictors=df.shape[1]-1

number_predictors_allowed=10
big_m=20




sigma,obj=generate_sigma_obj(df_train)
A,b,sense=prepare_constraint_matrix(K=number_predictors_allowed,M=big_m)
ub,lb,vtype=formulate_bounds_vtypes(M=big_m)
biMod=optimize_loss(A,b,sense,vtype,ub,lb)
err=compute_loss(biMod,df_test,return_weight=False)





